# Кибериммунная автономность $\\$Создание конструктивно защищённого автономного наземного транспортного средства $\\$Модуль 5


## Модуль 5. Раздельное принятие и применение решений о безопасности.


В современных системах управления автономными наземными транспортными средствами (АНТС) особое значение приобретают безопасность, управляемость и масштабируемость архитектуры. Для повышения уровня контроля над выполнением заданий и гибкости в управлении маршрутами необходимо внедрение дополнительных компонентов, обеспечивающих аутентификацию отправителей и централизованное управление маршрутной информацией.
Цель проекта - расширить существующую архитектуру системы управления автономным наземным транспортным средством (АНТС) путем внедрения двух новых компонентов:
- **Блок контроля доступа (Access Control Block)** – отвечает за проверку подлинности запросов, исходящих от отправителей заданий.
- **Менеджер ресурсов (Resource Manager)** – обеспечивает хранение и управление регулярными маршрутами, реализуя файловую систему маршрутов.

Новое решение должно гарантировать, что только уполномоченные отправители смогут активировать выполнение маршрутов, хранящихся в файловой системе, при этом в задание передаются лишь идентификаторы отправителя и маршрута.

В данном решении используется шаблон «Раздельное принятие и применение решений о безопасности» предназначенный для реализации активного механизма контроля доступа и фильтрации потоков данных/потоков управления на основе заданных правил и политик безопасности. Шаблон предполагает разделение механизмов принятия решения о возможности доступа или разрешении потока и применения этого решения к потокам данных, потокам управления и выполняемым в системе операциям. Это в конечном счете позволяет улучшить гибкость работы механизма контроля доступа и/или фильтрации потоков данных/потоков управления в системе и оптимизировать доказательство корректности его работы.

### Структура передачи заданий (миссий)

**Новый формат задания:**  
Вместо передачи полного объекта маршрута миссия должна содержать только идентификатор отправителя (`sender_id`) и идентификатор маршрута (`route_id`).

**Пример:**

```json
{
    "sender_id": "sender_01",
    "route_id": "route_001"
}
```


### Цепочка обработки

1. Входящий запрос (миссия) поступает в **CommunicationGateway**.
2. Сначала осуществляется **проверка через блок контроля доступа**.
3. При положительном результате запрос передаётся в **Менеджер Ресурсов** для извлечения полного маршрута.
4. После получения полного маршрута данные комбинируются с информацией об отправителе и передаются в соответствующие очереди для дальнейшей обработки (например, в `ControlSystem` и `SafetyBlock`).


### Блок контроля доступа (Access Control Block)

#### Приём запроса

Модуль должен принимать запросы, содержащие:

- `sender_id` – идентификатор отправителя
- `route_id` – идентификатор маршрута

#### Проверка прав доступа

Система должна содержать внутреннее хранилище или конфигурацию, в которой для каждого отправителя указаны разрешённые маршруты.

**Пример структуры данных:**

```python
allowed_mapping = {
    "sender_01": ["route_001", "route_005"],
    "sender_02": ["route_002", "route_003"],
    # ...
}
```

### Логика проверки

Метод `_process_access_request(sender_id, route_id)` должен:

- Получать список разрешённых маршрутов для данного отправителя.
- Вызывать метод `_forward_to_resource_manager(sender_it, route_id)`, если `route_id` присутствует в списке.
- Вызывать метод `_send_rejection(sender_it, route_id)`, если запрос не авторизован.

#### Логирование

При неудачной проверке система должна зафиксировать событие (логирование ошибки), указывая, что доступ для данного отправителя и маршрута запрещён.

---


In [1]:
from abc import ABC, abstractmethod
from multiprocessing import Process
from multiprocessing import Queue
from queue import Empty
from src.event_types import ControlEvent
from src.queues_dir import QueuesDirectory
from time import sleep
from src.config import DEFAULT_LOG_LEVEL, LOG_INFO, CRITICALITY_STR, LOG_ERROR
from src.config import ACCESS_CONTROL_QUEUE_NAME



class BaseAccessControlBlock(Process, ABC):
    """ Базовый класс для реализации контроля доступа """
    log_prefix = "[BASE ACCESS]"
    event_source_name = ACCESS_CONTROL_QUEUE_NAME
    events_q_name = event_source_name

    def __init__(self, queues_dir: QueuesDirectory, log_level=DEFAULT_LOG_LEVEL):
        super().__init__()
        self._queues_dir = queues_dir
        self.log_level = log_level

        # Инициализация очередей
        self._events_q = Queue()
        self._control_q = Queue()
        self._quit = False
        self._recalc_interval_sec = 0.1

        self._queues_dir.register(self._events_q, self.events_q_name)
        self._log_message(LOG_INFO, "Инициализирован базовый блок контроля доступа")

    def _log_message(self, criticality: int, message: str):
        if criticality <= self.log_level:
            print(f"[{CRITICALITY_STR[criticality]}]{self.log_prefix} {message}")

    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                self._quit = True
        except Empty:
            pass

    @abstractmethod
    def _process_access_request(self, sender_id: str, route_id: str):
        """ Основная логика обработки запроса (реализуется в наследниках) """
        pass

    def stop(self):
        self._control_q.put(ControlEvent(operation='stop'))

    def run(self):
        self._log_message(LOG_INFO, "Старт обработки запросов")
        while not self._quit:
            sleep(self._recalc_interval_sec)
            try:
                self._check_control_q()
                while True:
                    event = self._events_q.get_nowait()
                    params = event.parameters
                    self._log_message(LOG_INFO, f"перед вызовом self._process_access_request, sender_id: {params.get('sender_id')}, route_id: {params.get('route_id')}")
                    self._process_access_request(
                        params.get('sender_id'),
                        params.get('route_id')
                    )
            except Empty:
                continue
            except Exception as e:
                self._log_message(LOG_ERROR, f"Ошибка: {str(e)}")

In [2]:
from src.event_types import Event
from src.config import RESOURCE_MANAGER_QUEUE_NAME


class AccessControlBlock(BaseAccessControlBlock):

    def __init__(self, queues_dir: QueuesDirectory,
                 allowed_mapping: dict,  # Конфигурация разрешений
                 log_level=DEFAULT_LOG_LEVEL):
        super().__init__(queues_dir, log_level)
        self.allowed_mapping = allowed_mapping

    def _process_access_request(self, sender_id: str, route_id: str):
        self._log_message(LOG_INFO, f"в вызове self._process_access_request, sender_id: {sender_id}, route_id: {sender_id}")
        if not sender_id or not route_id:
            self._log_message(LOG_ERROR, "Некорректные параметры запроса")
            return

        if self._check_permissions(sender_id, route_id):
            self._log_message(LOG_INFO, f"перед вызовом forward_to_resource_manager, sender_id: {sender_id}, route_id: {sender_id}")
            self._forward_to_resource_manager(sender_id, route_id)
        else:
            self._send_rejection(sender_id, route_id, "access_denied")

    def _send_rejection(self, sender_id: str, route_id: str, reason: str):
        planner_q = self._queues_dir.get_queue(PLANNER_QUEUE_NAME)
        planner_q.put(Event(
            source=self.event_source_name,
            destination=PLANNER_QUEUE_NAME,
            operation="mission_rejected",
            parameters={
                "sender_id": sender_id,
                "route_id": route_id,
                "reason": reason
            }
        ))


    def _check_permissions(self, sender_id: str, route_id: str) -> bool:
        return route_id in self.allowed_mapping.get(sender_id, [])


    def _forward_to_resource_manager(self, sender_id: str, route_id: str):
        resource_q = self._queues_dir.get_queue(RESOURCE_MANAGER_QUEUE_NAME)
        print(resource_q)
        resource_q.put(Event(
            source=self.event_source_name,
            destination=RESOURCE_MANAGER_QUEUE_NAME,
            operation="take_mission",
            parameters={
                "sender_id": sender_id,
                "route_id": route_id
            }
        ))
        self._log_message(LOG_INFO, f"Маршрут {route_id} запрошен у ResourceManager")

## Менеджер ресурсов (Resource Manager)

#### Хранение маршрутов

Маршруты должны храниться:

- В виде отдельных файлов (например, `.json`).
- В демонстрационном варианте допустимо использование встроенного словаря.
- Для реального внедрения предпочтительно использовать файловую систему.

### Получение маршрута

На основании входящего `route_id` модуль должен:

- Искать соответствующий файл или запись в хранилище.
- Извлекать данные.
- Возвращать их в виде объекта (например, словаря Python).

### Обработка ошибок

Если маршрут не найден, модуль должен:

- Сгенерировать исключение или уведомить вызывающий код об отсутствии данных.
- Записать информацию в лог об ошибке (например, `"Маршрут с id route_001 не найден"`).

---

In [3]:
from abc import ABC, abstractmethod
from multiprocessing import Process
from src.config import RESOURCE_MANAGER_QUEUE_NAME
from multiprocessing import Queue
from queue import Empty
from src.event_types import ControlEvent
from src.queues_dir import QueuesDirectory
from time import sleep
from src.config import DEFAULT_LOG_LEVEL, LOG_INFO, CRITICALITY_STR, LOG_ERROR

class BaseResourceManager(Process, ABC):
    """ Базовый класс для управления ресурсами """
    log_prefix = "[BASE RESOURCE]"
    event_source_name = RESOURCE_MANAGER_QUEUE_NAME
    events_q_name = event_source_name

    def __init__(self, queues_dir: QueuesDirectory, log_level=DEFAULT_LOG_LEVEL):
        super().__init__()
        self._queues_dir = queues_dir
        self.log_level = log_level

        # Инициализация очередей
        self._events_q = Queue()
        print(self._events_q)
        self._control_q = Queue()
        self._quit = False
        self._recalc_interval_sec = 0.1

        self._queues_dir.register(self._events_q, self.events_q_name)
        self._log_message(LOG_INFO, "Инициализирован базовый менеджер ресурсов")

    def _log_message(self, criticality: int, message: str):
        if criticality <= self.log_level:
            print(f"[{CRITICALITY_STR[criticality]}]{self.log_prefix} {message}")

    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                self._quit = True
        except Empty:
            pass

    @abstractmethod
    def _get_route_data(self, route_id: str) -> dict:
        """ Получение данных маршрута (реализуется в наследниках) """
        pass

    def stop(self):
        self._control_q.put(ControlEvent(operation='stop'))

    @abstractmethod
    def _send_rejection(self, sender_id: str, route_id: str, reason: str):
        pass


    def run(self):
        self._log_message(LOG_INFO, "Старт обработки маршрутов")
        while not self._quit:
            sleep(self._recalc_interval_sec)
            try:
                self._check_control_q()

                try:
                    event = self._events_q.get_nowait()
                    self._log_message(LOG_INFO, f"после self._events_q.get_nowait")
                    # Проверка наличия обязательных параметров
                    if 'sender_id' not in event.parameters or 'route_id' not in event.parameters:
                        raise KeyError("Отсутствуют обязательные параметры sender_id или route_id")

                    route_id = event.parameters['route_id']
                    sender_id = event.parameters['sender_id']

                    try:
                        route_data = self._get_route_data(route_id)
                        self._dispatch_mission(route_data)
                        self._log_message(LOG_INFO, f"Маршрут {route_id} успешно отправлен")

                    except ValueError as e:
                        self._log_message(LOG_ERROR, str(e))
                        self._send_rejection(sender_id, route_id, "route_not_found")

                except KeyError as e:
                    self._log_message(LOG_ERROR, f"Некорректный запрос: {str(e)}")
                    self._send_rejection("unknown", "unknown", "invalid_request_format")

            except Empty:
                continue

            except Exception as e:
                self._log_message(LOG_ERROR, f"Критическая ошибка: {str(e)}")
                self._send_rejection("unknown", "unknown", "internal_server_error")


    @abstractmethod
    def _dispatch_mission(self, mission_data: dict):
        pass


In [4]:
from src.config import CONTROL_SYSTEM_QUEUE_NAME, SAFETY_BLOCK_QUEUE_NAME, PLANNER_QUEUE_NAME, LOG_DEBUG



class ResourceManager(BaseResourceManager):

    def __init__(self, queues_dir: QueuesDirectory,
                 routes_storage: dict,  # Хранилище маршрутов
                 log_level=DEFAULT_LOG_LEVEL):
        super().__init__(queues_dir, log_level)
        self.routes_storage = routes_storage


    def _get_route_data(self, route_id: str) -> dict:
        if route_id not in self.routes_storage:
            raise ValueError(f"Маршрут {route_id} не найден")
        return self.routes_storage[route_id]


    def _dispatch_mission(self, route_data: dict):
        control_q = self._queues_dir.get_queue(CONTROL_SYSTEM_QUEUE_NAME)
        safety_q = self._queues_dir.get_queue(SAFETY_BLOCK_QUEUE_NAME)

        mission = Mission(route_data["home"], route_data["waypoints"], route_data["speed_limits"], route_data["armed"])

        control_q.put(Event(
            source=self.event_source_name,
            destination=CONTROL_SYSTEM_QUEUE_NAME,
            operation="set_mission",
            parameters=mission
        ))

        safety_q.put(Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="set_mission",
            parameters=mission
        ))


    def _send_rejection(self, sender_id: str, route_id: str, reason: str):
        planner_q = self._queues_dir.get_queue(PLANNER_QUEUE_NAME)
        planner_q.put(Event(
            source=self.event_source_name,
            destination=PLANNER_QUEUE_NAME,
            operation="mission_rejected",
            parameters={
                "sender_id": sender_id,
                "route_id": route_id,
                "reason": reason
            }
        ))
        self._log_message(LOG_DEBUG, f"Отправлен отказ для {sender_id}:{route_id}. Причина: {reason}")


## Последовательность обработки запроса
Новая миссия содержит только два поля:
- `sender_id`
- `route_id`

### Проверка доступа
- Идентификаторы передаются в блок контроля доступа.
- Если проверка проходит не успешно:
  - Запрос отклоняется.
  - Происходит логирование ошибки.
- Если проверка проходит успешно, запрос передается на следующий этап.

### Загрузка маршрута

- Менеджер ресурсов получает `route_id`.
- Если маршрут найден:
  - Извлекается и возвращается.
- Если маршрут не найден:
  - Генерируется уведомление об ошибке.
  - Запрос отклоняется.

### Формирование и отправка миссии

- На основе данных проверки и маршрута формируется итоговый объект миссии.
- Объект передаётся в соответствующие очереди.
- Очереди назначения определяются конфигурацией:
  - `CONTROL_SYSTEM_QUEUE_NAME`
  - `SAFETY_BLOCK_QUEUE_NAME`
- Сообщения отправляются как экземпляры класса `Event`.

---

## Техническое взаимодействие с другими компонентами

### CommunicationGateway

- Был модернезирован метод `_send_mission_to_consumers`, чтобы он выполнял следующие действия:
  - Извлечение идентификаторов.
  - Проверка доступа.
  - Загрузка маршрута.
  - Формирование и отправка объекта миссии.

### ControlSystem и SafetyBlock

- Принимают событие с объектом миссии.
- Ожидаемые поля:
  - `route` — полный маршрут.
  - `sender_id` — идентификатор отправителя.
- Обработка выполняется стандартными алгоритмами.

---

In [5]:
from jupyter_events.validators import resources

# ваш код
from src.security_monitory import BaseSecurityMonitor
from src.security_policy_type import SecurityPolicy

from src.config import SERVOS_QUEUE_NAME, COMMUNICATION_GATEWAY_QUEUE_NAME, NAVIGATION_QUEUE_NAME, SAFETY_BLOCK_QUEUE_NAME, CONTROL_SYSTEM_QUEUE_NAME, CARGO_BAY_QUEUE_NAME, PLANNER_QUEUE_NAME, ACCESS_CONTROL_QUEUE_NAME, RESOURCE_MANAGER_QUEUE_NAME
from src.config import LOG_DEBUG, LOG_FAILURE, DEFAULT_LOG_LEVEL, LOG_INFO, LOG_ERROR

from multiprocessing import Queue
from src.event_types import Event
from src.mission_type import Mission
from src.safety_block import BaseSafetyBlock
import math

from src.communication_gateway import BaseCommunicationGateway
from src.control_system import BaseControlSystem
from src.access_control_block import BaseAccessControlBlock
from src.navigation_system import BaseNavigationSystem
from src.resource_manager import BaseResourceManager
from src.queues_dir import QueuesDirectory
    
    
class SecurityMonitor(BaseSecurityMonitor):
    """ класс монитора безопасности """

    def __init__(self, queues_dir):
        super().__init__(queues_dir)
        self._init_set_security_policies()

    def _init_set_security_policies(self):
        """ инициализация политик безопасности """
        default_policies = [
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='set_mission'
            ),
            SecurityPolicy(
                source=PLANNER_QUEUE_NAME,
                destination=ACCESS_CONTROL_QUEUE_NAME,
                operation='take_mission'
            ),
            SecurityPolicy(
                source=ACCESS_CONTROL_QUEUE_NAME,
                destination=PLANNER_QUEUE_NAME,
                operation="mission_rejected",
            ),
            SecurityPolicy(
                source=ACCESS_CONTROL_QUEUE_NAME,
                destination=RESOURCE_MANAGER_QUEUE_NAME,
                operation="take_mission",
            ),
            SecurityPolicy(
                source=RESOURCE_MANAGER_QUEUE_NAME,
                destination=PLANNER_QUEUE_NAME,
                operation="set_mission",
            ),
            SecurityPolicy(
                source=RESOURCE_MANAGER_QUEUE_NAME,
                destination=PLANNER_QUEUE_NAME,
                operation="mission_rejected",
            ),
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_mission'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_speed'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_speed'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_direction'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_direction'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='release_cargo'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='release_cargo'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='lock_cargo'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='lock_cargo'
            ),
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='position_update'
            ),
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='position_update'
            ),
        ]
        self.set_security_policies(policies=default_policies)        

    def set_security_policies(self, policies):
        """ установка новых политик безопасности """
        self._security_policies = policies
        self._log_message(LOG_INFO, f"изменение политик безопасности: {policies}")

    def _check_event(self, event: Event):
        """ проверка входящих событий """
        self._log_message(LOG_DEBUG, f"проверка события {event}, по умолчанию выполнение запрещено")

        request = SecurityPolicy(
            source=event.source,
            destination=event.destination,
            operation=event.operation)

        if request in self._security_policies:
            self._log_message(LOG_DEBUG, "событие разрешено политиками, выполняем")
            return True

        self._log_message(LOG_ERROR, f"событие не разрешено политиками безопасности! {event}")
        return False


class SafetyBlock(BaseSafetyBlock):
    """ класс ограничений безопасности """

    def _set_new_direction(self, direction: float):
        """ установка нового направления перемещения """
        self._log_message(LOG_INFO, f"текущие координаты: {self._position}")
        self._log_message(LOG_DEBUG, f"маршрутное задание: {self._mission}")
        self._log_message(LOG_DEBUG, f"состояние маршруте: {self._route}")
        
        if not self._mission or not self._position or not self._route:
            self._log_message(LOG_ERROR, "Неизвестный маршрут")
            self._stop_afcs()
            return
        
        if not self._route.next_point():
            self._log_message(LOG_FAILURE, "Нет следующей точки маршрута.")
            self._stop_afcs()
            return
        
        next_direction = self._calculate_direction()
        error_direction = 1

        if (abs(next_direction - direction) > error_direction):
            self._log_message(LOG_ERROR, "Внештатное изменение направления движения.")

        self._direction = next_direction
        self._send_direction_to_consumers()

    def _set_new_speed(self, speed: float):
        """ установка новой скорости """
        if not self._mission or not self._position or not self._route:
            self._log_message(LOG_ERROR, "Неизвестный маршрут")
            self._stop_afcs()
            return

        allowed_speed = self._route.calculate_speed()
        if allowed_speed < speed:
            self._log_message(LOG_ERROR, f"Внештатное превышение скорости: {speed}. Разрешенная скорость: {allowed_speed}")
            self._speed = allowed_speed
        else:
            self._speed = speed
        
        self._send_speed_to_consumers()

    def _send_speed_to_consumers(self):
        self._log_message(LOG_DEBUG, "отправляем скорость получателям")
        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_speed",
                            parameters=self._speed
                            )
        servos_q.put(event_speed)

    def _send_direction_to_consumers(self):
        self._log_message(LOG_DEBUG, "отправляем направление получателям")

        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_direction",
                            parameters=self._direction
                            )
        servos_q.put(event_speed)

    def _lock_cargo(self, _):
        self._log_message(LOG_INFO, "Блокировка грузового отсека.")
        self._send_lock_cargo_to_consumers()
    
    def _release_cargo(self, _):
        if not self._route.route_finished:
            self._log_message(LOG_FAILURE, "АНТС не в конечной точке маршрута. Невозможно разблокировать грузовой отсек.")
            return
        self._send_release_cargo_to_consumers()

    def _send_lock_cargo_to_consumers(self):
        self._log_message(LOG_INFO, "Блокировка грузового отсека")
        cargo_q_name = CARGO_BAY_QUEUE_NAME
        cargo_q = self._queues_dir.get_queue(cargo_q_name)
        # инициализация сообщения с командой на блокировку грузового отсека
        event = Event(
            source=self.event_source_name,
            destination=cargo_q_name,
            operation="lock_cargo",
            parameters=None
        )
        cargo_q.put(event)
        
    
    def _send_release_cargo_to_consumers(self):
        self._log_message(LOG_INFO, "Разблокировка грузового отсека")
        cargo_q_name = CARGO_BAY_QUEUE_NAME
        cargo_q = self._queues_dir.get_queue(cargo_q_name)
        # инициализация сообщения с командой на разблокировку грузового отсека
        event = Event(
            source=self.event_source_name,
            destination=cargo_q_name,
            operation="release_cargo",
            parameters=None
        )
        cargo_q.put(event)
    
    def _stop_afcs(self):
        self._direction = 0
        self._speed = 0
        self._log_message(LOG_FAILURE, "АНТС принудительно остановлено.")
    
    def _calculate_direction(self):
        current_lattitude = math.radians(self._position.latitude)
        next_lattitude = math.radians(self._route.next_point().latitude)
        delta_longtitude = math.radians(self._route.next_point().longitude - self._position.longitude)

        x = math.sin(delta_longtitude) * math.cos(next_lattitude)
        y = math.cos(current_lattitude) * math.sin(next_lattitude) - \
            math.sin(current_lattitude) * math.cos(next_lattitude) * math.cos(delta_longtitude)

        return (math.degrees(math.atan2(x, y)) + 360) % 360


class CommunicationGateway(BaseCommunicationGateway):
    """CommunicationGateway класс для реализации логики взаимодействия
    с системой планирования заданий

    Работает в отдельном процессе, поэтому создаётся как наследник класса Process
    """
    def _send_mission_to_consumers(self):
        """ метод для отправки сообщения с маршрутным заданием в систему управления """
        
        # имена очередей блоков находятся в файле src/config.py
        # события нужно отправлять в соответствие с диаграммой информационных потоков
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME

        # события передаются в виде экземпляров класса Event, 
        # описание класса находится в файле src/event_types.py
        event = Event(source=BaseCommunicationGateway.event_source_name,
                      destination=control_q_name,
                      operation="set_mission", parameters=self._mission
                      )
        
        safety_event = Event(source=BaseCommunicationGateway.event_source_name,
                      destination=safety_q_name,
                      operation="set_mission", parameters=self._mission
                      )

        # поиск в каталоге нужной очереди (в данном случае - системы управления)
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        safety_q: Queue = self._queues_dir.get_queue(safety_q_name)
        # отправка события в найденную очередь
        control_q.put(event)
        safety_q.put(safety_event)


class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """

    def _send_speed_and_direction_to_consumers(self, speed, direction):
        # servos_q_name = SERVOS_QUEUE_NAME
        # servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        safety_q_name = SAFETY_BLOCK_QUEUE_NAME
        safety_q: Queue = self._queues_dir.get_queue(safety_q_name)

        # инициализация сообщения с желаемой скоростью
        event_speed = Event(
            source=self.event_source_name,
            destination=safety_q_name,
            operation="set_speed",
            parameters=speed
        )

        # отправка сообщения с желаемым направлением
        event_direction = Event(
            source=self.event_source_name,
            destination=safety_q_name,
            operation="set_direction",
            parameters=direction
        )

        safety_q.put(event_speed)
        safety_q.put(event_direction)
    
    def _lock_cargo(self):
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME
        safety_q = self._queues_dir.get_queue(safety_q_name)
        # инициализация сообщения с командой на блокировку грузового отсека
        event = Event(
            source=self.event_source_name,
            destination=safety_q_name,
            operation="lock_cargo",
            parameters=None
        )
        safety_q.put(event)

    def _release_cargo(self):
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME
        safety_q = self._queues_dir.get_queue(safety_q_name)
        # инициализация сообщения с командой на разблокировку грузового отсека
        event = Event(
            source=self.event_source_name,
            destination=safety_q_name,
            operation="release_cargo",
            parameters=None
        )
        safety_q.put(event)


class NavigationSystem(BaseNavigationSystem):
    """ класс навигационного блока """
    def _send_position_to_consumers(self):        
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME

        event = Event(
            source=self.event_source_name,
            destination=control_q_name,
            operation="position_update",
            parameters=self._position
        )

        safety_event = Event(
            source=self.event_source_name,
            destination=safety_q_name,
            operation="position_update",
            parameters=self._position
        )

        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        safety_q: Queue = self._queues_dir.get_queue(safety_q_name)
        control_q.put(event)
        safety_q.put(safety_event)

In [6]:
# используем то же маршрутное задание, которое было в модуле 2
from time import sleep

from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.mission_planner import Mission
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer
from src.wpl_parser import WPLParser
from src.mission_type import GeoSpecificSpeedLimit
afcs_present = True
car_id = "m5"


# возьмём маршрут из модуля 2
wpl_file = "route-module4.wpl"

parser = WPLParser(wpl_file)    
points = parser.parse()


# обновите скоростные ограничения для вашего маршрута!
speed_limits = [
    GeoSpecificSpeedLimit(0, 30),
    GeoSpecificSpeedLimit(10, 40),
    GeoSpecificSpeedLimit(22, 60),
    GeoSpecificSpeedLimit(49, 30),
    GeoSpecificSpeedLimit(52, 90),
    GeoSpecificSpeedLimit(53, 40),
    GeoSpecificSpeedLimit(73, 90),
    GeoSpecificSpeedLimit(99, 40),
    GeoSpecificSpeedLimit(111, 60),
]

home = points[0]

ROUTES_STORAGE = {
    "route_1": {
        "home": points[0],
        "waypoints": points,
        "speed_limits": speed_limits,
        "armed": True,
    }
}

ALLOWED_MAPPING = {
    "sender_1": "route_1"
}

# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 


# создание блоков передачи данных в СУПА
if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

# создание основных функциональных блоков

access_control_block = AccessControlBlock(
    queues_dir, allowed_mapping=ALLOWED_MAPPING
)

resource_manager = ResourceManager(
    queues_dir=queues_dir, routes_storage=ROUTES_STORAGE
)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)

communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

safety_block = SafetyBlock(queues_dir=queues_dir, log_level=LOG_INFO)
security_monitor = SecurityMonitor(queues_dir=queues_dir)

mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, sender_id="sender_1", route_id="route_1")

# сборка всех запускаемых блоков в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        access_control_block,
        resource_manager,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security_monitor
    ] if afcs_present else [
        sitl,
        mission_planner,
        access_control_block,
        resource_manager,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security_monitor
    ])

#################################
# АКТИВАЦИЯ КИБЕРПРЕПЯТСТВИЙ
control_system.enable_surprises()
#################################

# запуск всех блоков
system_components.start()

# ограничение поездки по времени
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(1000)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь access_control
[ИНФО][BASE ACCESS] Инициализирован базовый блок контроля доступа
[ИНФО][QUEUES] регистрируем очередь resource
[ИНФО][BASE RESOURCE] Инициализирован базовый менеджер ресурсов
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][QUEUES] регистрируем очередь safety
[ИНФО][SAFETY] создан ограничитель
[ИНФО][QUEUES] регистрируем очередь security
[ИНФО][SECURITY] создан монитор безопасности
[ИНФО][SECURITY] изменение политик безопасности: [SecurityPolicy(source='communication', desti